In [2]:
import pandas as pd
import os
import numpy as np


In [9]:
dir = (r'C:\Users\daryl\OneDrive\Documents\GDAA3000\ProjectDischarge\RdrsSample'
       r'\LstmDatasets\NsWeatherData\01ED007')
q_dir = (r'C:\Users\daryl\OneDrive\Documents\GDAA3000\ProjectDischarge'
         r'\RdrsSample\LstmDatasets\basin_q')

filenames = os.listdir(dir)
q_files = os.listdir(q_dir)

for file in filenames:
    basin_id = file.split('_')[0]
    if file.endswith('01ED007_weather_hourly-daily_joined_2011-2021_daily.csv'):
        stn = os.path.join(dir, file)
        df_stn = pd.read_csv(stn)
        # Convert 'date' column to a datetime object
        df_stn['date'] = pd.to_datetime(df_stn['date'])
        df_stn['date'] = df_stn['date'].dt.strftime('%Y/%m/%d')
        df_stn['date'] = pd.to_datetime(df_stn['date'])
        # print(df_stn.head())
        df_stn = df_stn.rename(columns={'max_tem_deg_c': 'max_temp_deg_c'})
        print(df_stn.columns)


        for q in q_files:
            if q.endswith('01ED007_daily_20240507T2314.csv'):
                q_csv = os.path.join(q_dir, q)
                q_basin_id = q.split('_')[0]
                if basin_id == q_basin_id:
                    df_q = pd.read_csv(q_csv, parse_dates=['Date'], skiprows=1)
                    # Rename the 'Date' column to 'date' in df_q.
                    df_q.rename(columns={'Date': 'date'}, inplace=True)
                    df_q['date'] = df_q['date'].dt.strftime('%Y/%m/%d')
                    # Convert 'date' column to a datetime object
                    df_q['date'] = pd.to_datetime(df_q['date'])
                    # Make 'date' the index of df_q.
                    # df_q.set_index('date', inplace=True)
                    # print(df_q)
                    ##### Filter to desired date range #############################
                    # Find the earliest and latest dates in df_stn and use them to filter .
                    start_date = df_stn['date'].min()
                    end_date = df_stn['date'].max()
                    
                    # Filter df_q based on the date range
                    df_q = df_q[(df_q['date'] >= start_date) & (df_q['date'] <= end_date)]
                    ##### Reorganize the data ######################################
                    df_q_filt_stage = df_q[df_q['PARAM'] == 1]
                    # Filter df_q to only include rows where 'PARAM'== 2 (discharge rate).
                    df_q_filt_q = df_q[df_q['PARAM'] == 2]
                    # For df_q_filt_stage, rename the 'Value' column to 'stage_m' and drop the 'PARAM' column.
                    df_q_filt_stage.rename(columns={'Value': 'stage_m'}, inplace=True)
                    df_q_filt_stage.drop(columns=['PARAM'], inplace=True)
                    # For df_q_filt_q, rename the 'Value' column to 'q_m3_s' and drop the 'PARAM' column.
                    df_q_filt_q.rename(columns={'Value': 'q_m3_s'}, inplace=True)
                    df_q_filt_q.drop(columns=['PARAM'], inplace=True)
                    # print(df_q_filt_q)
                    # Merge df_q_filt_stage and df_q_filt_q on 'date' using an inner join.
                    df_q_joined = pd.merge(df_q_filt_stage, df_q_filt_q, on='date', how='inner')
                    # Interpolate missing values in  for columns 'q_m3_s'.
                    df_q_joined['q_m3_s'] = df_q_joined['q_m3_s'].interpolate()
                    df_q_joined['stage_m'] = df_q_joined['stage_m'].interpolate()
                    # Drop unnecessary columns from df_daymet. Warning: there is a space in the ' ID' 
                    # and ' ID_y' column names.
                    df_q_joined.drop(columns=[ ' ID_x', ' ID_y', 'SYM_x', 'SYM_y'], inplace=True)
                    # print(df_q_joined.columns)
                    # print(df_q_joined)

                    # # Set the date as the index
                    df_q_joined.set_index('date', inplace=True)

                    # Check if the index is regularly spaced
                    print(df_q_joined.index.freq)

                    # Make the index regularly spaced (e.g., daily)
                    df_q_joined = df_q_joined.asfreq('D')

                    # Check if the index is regularly spaced
                    print(df_q_joined.index.freq)
                    # Print the first and last dates of the index
                    print(df_q_joined.index[0], df_q_joined.index[-1])
                    # print(df_q_joined.head())
                    # print(df_stn.head())
                    # Merge df_daymet and df_q_joined on 'date' using an inner join.
                    df_q_joined = pd.merge(df_stn, df_q_joined, on='date', how='inner')
                    # Change date format to 'dd/mm/yyyy'.
                    
                    df_q_joined = df_q_joined.rename(columns={'dew_point_C': 'dew_point_deg_c'})
                    df_q_joined.replace('', np.nan, inplace=True)
                    # print(df_q_joined.columns)
                    output_dir = (r'C:\Users\daryl\OneDrive\Documents\GDAA3000'
                                  r'\ProjectDischarge\RdrsSample\LstmDatasets\JoinedQWeatherStn')
                    # df_q_joined_dropped = df_q_joined.drop(columns=['Id'])
                
                    # Get a list of all the columns
                    cols = list(df_q_joined.columns)
                    # print(cols)
                    # Remove 'date' from the list
                    cols.remove('date')
                    # Reorder the columns to make 'date' the first column
                    df_q_joined_dropped = df_q_joined[['date'] + cols]
                    print(df_q_joined_dropped.columns)
                    # Interpolate missing values in df_q_joined_dropped
                    df_q_joined_dropped['max_temp_deg_c'] = df_q_joined_dropped['max_temp_deg_c'].interpolate()
                    df_q_joined_dropped['min_temp_deg_c'] = df_q_joined_dropped['min_temp_deg_c'].interpolate()
                    df_q_joined_dropped['mean_temp_deg_c'] = df_q_joined_dropped['mean_temp_deg_c'].interpolate()
                    df_q_joined_dropped['q_m3_s'] = df_q_joined_dropped['q_m3_s'].interpolate()
                    df_q_joined_dropped['dew_point_deg_c'] = df_q_joined_dropped['dew_point_deg_c'].interpolate()
                    df_q_joined_dropped['rel_hum_percent'] = df_q_joined_dropped['rel_hum_percent'].interpolate()
                    # Set the date as the index
                    df_q_joined_dropped.set_index('date', inplace=True)

                    # Check if the index is regularly spaced
                    print(df_q_joined_dropped.index.freq)

                    # Make the index regularly spaced (e.g., daily)
                    df_q_joined_dropped = df_q_joined_dropped.asfreq('D')

                    # Check if the index is regularly spaced
                    print(df_q_joined_dropped.index.freq)
                    # Save the joined dataframe as a csv file in the output_dir.
                    df_q_joined_dropped.to_csv(os.path.join(output_dir, f'{q_basin_id}_stn_2011-2021_q_new.csv'))
                    print(df_q_joined_dropped)

                    

                                   

Index(['date', 'Climate ID', 'max_temp_deg_c', 'min_temp_deg_c',
       'mean_temp_deg_c', 'heat_deg_days_deg_c', 'cool_deg_days_deg_c',
       'rain_mm', 'snow_cm', 'total_prcp_mm', 'snow_on_grnd_cm',
       'dir_max_gust_10s_deg', 'spd_max_gust_km_h', 'dew_point_C',
       'rel_hum_percent'],
      dtype='object')
None
<Day>
2011-01-01 00:00:00 2021-12-31 00:00:00
Index(['date', 'Climate ID', 'max_temp_deg_c', 'min_temp_deg_c',
       'mean_temp_deg_c', 'heat_deg_days_deg_c', 'cool_deg_days_deg_c',
       'rain_mm', 'snow_cm', 'total_prcp_mm', 'snow_on_grnd_cm',
       'dir_max_gust_10s_deg', 'spd_max_gust_km_h', 'dew_point_deg_c',
       'rel_hum_percent', 'stage_m', 'q_m3_s'],
      dtype='object')
None
<Day>
            Climate ID  max_temp_deg_c  min_temp_deg_c  mean_temp_deg_c  \
date                                                                      
2011-01-01     8202000             6.3             0.1              3.2   
2011-01-02     8202000             1.9            -1

C:\Users\daryl\AppData\Local\Temp\ipykernel_6644\236312825.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_q_filt_stage.rename(columns={'Value': 'stage_m'}, inplace=True)
C:\Users\daryl\AppData\Local\Temp\ipykernel_6644\236312825.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_q_filt_stage.drop(columns=['PARAM'], inplace=True)
C:\Users\daryl\AppData\Local\Temp\ipykernel_6644\236312825.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

In [ ]:
                    #### Check for missing values ################################
                    Identify groups of consecutive missing values
                    df_q['group'] = df_q['Value'].isna().ne(df_q['Value'].shift().isna()).cumsum()

                    # Count the number of missing values in each group
                    counts = df_q.groupby('group')['Value'].apply(lambda x: x.isna().sum())

                    # Calculate the total number of rows for groups with more than 10 consecutive missing values
                    total_rows = counts[counts > 10].sum()

                    # Calculate 10% of the total original number of rows in df_q
                    ten_percent = df_q.shape[0] * 0.1

                    # If the total number of rows for groups with more than 10 consecutive missing values is greater than 10% of the total original number of rows in df_q
                    if total_rows > ten_percent:
                        # Calculate the percentage of the df_q that is groups with more than 10 consecutive missing values
                        percent = (total_rows / df_q.shape[0]) * 100
                        # print(f"{percent}% of {q_basin_id} is groups with more than 10 consecutive missing values.")
                        break
                    elif total_rows <= ten_percent:
                        # impute missing values in df_q
                        df_q['Value'] = df_q['Value'].interpolate()

                    # Print the maximum number of consecutive missing values in the 'Value' column
                    print(f"Max number of consecutive missing values in {q_basin_id}: {counts.max()}")
                    # Print the percent of missing values in the 'Value' column.
                    print(f"Percent of missing values in {q_basin_id}: {df_q['Value'].isna().mean() * 100}")
                    print()
                    Convert missing values to NaN in df_q_joined